In [ ]:
import os
import geopandas as gpd
import rasterio
import rasterio.mask
import rasterio.warp
import numpy as np
from shapely.geometry import mapping

def sanitize_field_name(field_name, existing_fields):
    """确保字段名符合 shapefile 命名规则，并保持唯一性"""
    field_name = field_name.replace(".tif", "")  # 移除.tif
    field_name = field_name.replace("_", "")  # 移除下划线
    if field_name.startswith("BurnedArea"):
        year = field_name.replace("BurnedArea", "")
        field_name = f"BA_{year}"  # 改为 BA_年份 格式
    field_name = field_name[:10]  # 限制长度最多 10 个字符
    
    # 确保字段名唯一
    count = 1
    original_field = field_name
    while field_name in existing_fields:
        field_name = f"{original_field[:8]}{count}"  # 保持 10 字符限制
        count += 1
    existing_fields.add(field_name)
    return field_name

def calculate_burned_area(grid_shapefile, burned_area_folder, output_shapefile):
    # 读取格网 shapefile 并确保使用其坐标系
    grid = gpd.read_file(grid_shapefile)
    grid_crs = grid.crs  # 获取格网的坐标系
    
    existing_fields = set(grid.columns)  # 记录已有字段，防止重复
    
    # 遍历 BurnedArea 目录中的所有 tif 文件
    for tif_file in sorted(os.listdir(burned_area_folder)):
        if tif_file.endswith(".tif"):
            tif_path = os.path.join(burned_area_folder, tif_file)
            
            # 读取栅格数据并转换为与 shp 一致的坐标系
            with rasterio.open(tif_path) as src:
                if src.crs != grid_crs:
                    transform, width, height = rasterio.warp.calculate_default_transform(
                        src.crs, grid_crs, src.width, src.height, *src.bounds)
                    kwargs = src.meta.copy()
                    kwargs.update({"crs": grid_crs, "transform": transform, "width": width, "height": height})
                    
                    reprojected_tif = f"temp_{tif_file}"
                    with rasterio.open(reprojected_tif, "w", **kwargs) as dst:
                        for i in range(1, src.count + 1):
                            rasterio.warp.reproject(
                                source=rasterio.band(src, i),
                                destination=rasterio.band(dst, i),
                                src_transform=src.transform,
                                src_crs=src.crs,
                                dst_transform=transform,
                                dst_crs=grid_crs,
                                resampling=rasterio.enums.Resampling.nearest)
                else:
                    reprojected_tif = tif_path
                
                field_name = sanitize_field_name(tif_file, existing_fields)  # 处理字段名，确保唯一
                print(f"Adding field: {field_name}")  # 调试输出，确保字段正确
                grid[field_name] = 0  # 初始化属性字段
                
                with rasterio.open(reprojected_tif) as src_reprojected:
                    pixel_area_km2 = (src_reprojected.res[0] * src_reprojected.res[1]) / 1e6  # 计算像素面积，转换为 km²
                    for idx, row in grid.iterrows():
                        geom = [mapping(row.geometry)]  # 获取格网的几何
                        
                        try:
                            # 裁剪栅格
                            out_image, out_transform = rasterio.mask.mask(src_reprojected, geom, crop=True, nodata=0)
                            out_image = out_image[0]  # 只取第一个波段
                            
                            # 计算 Burned Area（值在 1-370 之间的像素数）
                            burned_area_km2 = np.count_nonzero((out_image >= 1) & (out_image <= 370)) * pixel_area_km2
                            grid.at[idx, field_name] = burned_area_km2
                        except Exception as e:
                            print(f"Warning: Unable to process {tif_file} for grid {idx}: {e}")
    
    # 保存新的 shapefile
    grid.to_file(output_shapefile)
    print(f"Updated shapefile saved to {output_shapefile}")

# 设定输入参数
grid_shp = "/your_path_here/grid5.shp"  # 请替换为你的格网文件路径
burned_area_folder = "/your_path_here/BurnedArea"  # 请替换为BurnedArea文件夹路径
output_shp = "/your_path_here/grid5_BurnedArea.shp"  # 请替换为输出文件路径

# 运行计算函数
calculate_burned_area(grid_shp, burned_area_folder, output_shp)

In [ ]:
import os
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
import rasterio.warp

def calculate_precipitation(grid_shapefile, pre_folder, output_shapefile):
    # 读取格网 shapefile
    grid = gpd.read_file(grid_shapefile)
    grid_crs = grid.crs  # 获取格网的坐标系
    
    # 遍历 Pre 目录中的所有 tif 文件
    for tif_file in sorted(os.listdir(pre_folder)):
        if tif_file.endswith(".tif"):
            tif_path = os.path.join(pre_folder, tif_file)
            
            # 读取栅格数据并确保坐标系一致
            with rasterio.open(tif_path) as src:
                if src.crs != grid_crs:
                    transform, width, height = rasterio.warp.calculate_default_transform(
                        src.crs, grid_crs, src.width, src.height, *src.bounds)
                    kwargs = src.meta.copy()
                    kwargs.update({"crs": grid_crs, "transform": transform, "width": width, "height": height})
                    
                    reprojected_tif = f"temp_{tif_file}"
                    with rasterio.open(reprojected_tif, "w", **kwargs) as dst:
                        for i in range(1, src.count + 1):
                            rasterio.warp.reproject(
                                source=rasterio.band(src, i),
                                destination=rasterio.band(dst, i),
                                src_transform=src.transform,
                                src_crs=src.crs,
                                dst_transform=transform,
                                dst_crs=grid_crs,
                                resampling=rasterio.enums.Resampling.nearest)
                else:
                    reprojected_tif = tif_path
                
                field_name = tif_file.replace(".tif", "").replace("_", "")[:10]  # 处理字段名，确保不超过 10 字符
                print(f"Adding field: {field_name}")  # 调试输出，确保字段正确
                
                # 计算格网内的降水均值
                stats = zonal_stats(grid, reprojected_tif, stats=["mean"], nodata=0)
                grid[field_name] = [s["mean"] if s["mean"] is not None else 0 for s in stats]
    
    # 保存新的 shapefile
    grid.to_file(output_shapefile)
    print(f"Updated shapefile saved to {output_shapefile}")

# 设定输入参数
grid_shp = "/your_path_here/grid5_BurnedArea.shp"  # 请替换为你的格网文件路径
pre_folder = "/your_path_here/Pre"  # 请替换为 Pre 文件夹路径
output_shp = "/your_path_here/grid5_Pre.shp"  # 请替换为输出文件路径
# 运行计算函数
calculate_precipitation(grid_shp, pre_folder, output_shp)

In [ ]:
import os
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
import rasterio.warp

def calculate_temperature(grid_shapefile, tem_folder, output_shapefile):
    # 读取格网 shapefile
    grid = gpd.read_file(grid_shapefile)
    grid_crs = grid.crs  # 获取格网的坐标系
    
    # 遍历 Tem 目录中的所有 tif 文件
    for tif_file in sorted(os.listdir(tem_folder)):
        if tif_file.endswith(".tif"):
            tif_path = os.path.join(tem_folder, tif_file)
            
            # 读取栅格数据并确保坐标系一致
            with rasterio.open(tif_path) as src:
                if src.crs != grid_crs:
                    transform, width, height = rasterio.warp.calculate_default_transform(
                        src.crs, grid_crs, src.width, src.height, *src.bounds)
                    kwargs = src.meta.copy()
                    kwargs.update({"crs": grid_crs, "transform": transform, "width": width, "height": height})
                    
                    reprojected_tif = f"temp_{tif_file}"
                    with rasterio.open(reprojected_tif, "w", **kwargs) as dst:
                        for i in range(1, src.count + 1):
                            rasterio.warp.reproject(
                                source=rasterio.band(src, i),
                                destination=rasterio.band(dst, i),
                                src_transform=src.transform,
                                src_crs=src.crs,
                                dst_transform=transform,
                                dst_crs=grid_crs,
                                resampling=rasterio.enums.Resampling.nearest)
                else:
                    reprojected_tif = tif_path
                
                field_name = tif_file.replace(".tif", "").replace("_", "")[:10]  # 处理字段名，确保不超过 10 字符
                print(f"Adding field: {field_name}")  # 调试输出，确保字段正确
                
                # 计算格网内的气温均值
                stats = zonal_stats(grid, reprojected_tif, stats=["mean"], nodata=0)
                grid[field_name] = [s["mean"] if s["mean"] is not None else 0 for s in stats]
    
    # 保存新的 shapefile
    grid.to_file(output_shapefile)
    print(f"Updated shapefile saved to {output_shapefile}")

# 设定输入参数
grid_shp = "/your_path_here/grid5_Pre.shp"  # 请替换为你的格网文件路径
tem_folder = "/your_path_here/Tem"  # 请替换为 Tem 文件夹路径
output_shp = "/your_path_here/grid5_Sta.shp"  # 请替换为输出文件路径

# 运行计算函数
calculate_temperature(grid_shp, tem_folder, output_shp)
